In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from __future__ import print_function
from functools import reduce
import re
import sys
import keras
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Merge, Dropout, RepeatVector
from keras.layers import recurrent
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from collections import OrderedDict
from keras.models import model_from_json
import h5py
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.models import load_model


import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


['train-data', 'testfile', 'dilton']


In [22]:
data = open('../input/train-data/train_LSTM_26112016.txt',"r")
print(data)

<_io.TextIOWrapper name='../input/train-data/train_LSTM_26112016.txt' mode='r' encoding='UTF-8'>


In [63]:
test = open('../input/testfile/test.txt',"r")
print(test)


<_io.TextIOWrapper name='../input/testfile/test.txt' mode='r' encoding='UTF-8'>


In [17]:
for line in data:
    print (line)

1 There were a total of num1 soccer games during the num3 month season.

2 If the games are equally divided, how many soccer games are played a month ?	num1 num3 /	1

1 Jill invited num3 people to her birthday party.

2 They each ate num2 pieces of pizza.

3 How many pieces of pizza did they eat?	num3 num2 *	1 2

1 Tim found num3 seashells and Sally found num1 seashells on the beach .

2 When they cleaned them , they discovered that 25 were cracked .

3 How many seashells did they find together ?	num3 num1 +	1 2

1 Kimberly went to the store num3 times last month.

2 She buys num2 peanuts each time she goes to the store.

3 How many peanuts did Kimberly buy last month?	num3 num2 *	1 2

1 Wanda has num3 blocks.

2 Theresa gives Wanda num1 more.

3 How many blocks does Wanda have in all?	num3 num1 +	1 2

1 Ezra drew a white line that was num1 inches long .

2 Then he drew a blue line that was num3 inches long .

3 How much longer was the white line than the blue line ?	num1 num3 -	1 2

1

3 How many quarters does she have now ?	num1 num2 -	1 2

1 Timothy starts with $num1 and spends $num3 on stickers.

2 How much money does Timothy have left ?	num1 num3 -	1

1 Paul had num2 strawberries in his basket.

2 He picked num3 more strawberries.

3 How many strawberries did he have then?	num2 num3 +	1 2

1 The library is divided into different sections for different type of books.

2 The science fiction section has num3 books.

3 If each book has 47num3 pages, how many pages do all the books have in total?	num3 num2 *	1 2

1 Eugene has num2 pencils.

2 He gets num1 more from Joyce.

3 How many pencils does Eugene have in all?	num2 num1 +	1 2

1 In Mr. Olsen 's mathematics class , num2 the students received A 's and num3 received B 's .

2 What fraction of the students received either A 's or B 's ?	num2 num3 +	1

1 Marta picked two pumpkins.

2 The first pumpkin weighed num3 pounds, and the second pumpkin weighed num1 pounds.

3 How much did the two pumpkins weigh all together?

In [23]:
def extract(data):
    dicto={}
    idx=-1
    l=[]
    for line in data:
        if(line[0]=="1"):
            idx=idx+1
            if(idx!=0):
                dicto[idx]=(l)
            l=[]
        l.append(line)
        
    return dicto
val = extract(data)

In [24]:
def extract_query(dicton):
    query=[]
    statement=[]
    for i in range(0,len(dicton)):
        query.append(dicton[i+1][-1])
        statement.append(dicton[i+1][:-1])
    return query,statement
query,statement = extract_query(val)


In [25]:
def clean_data(lst):
    state_list=[]
    for idx in range(0,len(lst)):
        state_list.append(' '.join(lst[idx]))
    return state_list
state_list = clean_data(statement)
print(state_list)

['1 There were a total of num1 soccer games during the num3 month season.\n', '1 Jill invited num3 people to her birthday party.\n 2 They each ate num2 pieces of pizza.\n', '1 Tim found num3 seashells and Sally found num1 seashells on the beach .\n 2 When they cleaned them , they discovered that 25 were cracked .\n', '1 Kimberly went to the store num3 times last month.\n 2 She buys num2 peanuts each time she goes to the store.\n', '1 Wanda has num3 blocks.\n 2 Theresa gives Wanda num1 more.\n', '1 Ezra drew a white line that was num1 inches long .\n 2 Then he drew a blue line that was num3 inches long .\n', '1 Sarah sold num3 boxes of Lemon Chalet Cremes.\n', '1 Joshua has num3 Skittles and 6 eggs.\n', '1 Willie weighs num2 pounds.\n 2 Kevin weighs num1 pounds.\n', '1 Each ticket costs $num20.\n', '', '1 There are num3 birds on the fence.\n 2 num1 more birds land on the fence.\n', '1 Jack is inviting num3 friends to a party.\n 2 He has num3 cookies.\n', "1 Alyssa's high school played n

In [26]:

stop_words1=['1','2','3','4','5','6','7','8','9','0','.',',']
def processing(lst):
    filtered_sentence=[[]]
    for i in range(0,len(lst)):
        word_tokens = word_tokenize(lst[i])
        filtered_sentence .append([w for w in word_tokens if not w in stop_words1])
    filtered_sentence.remove([])
    return filtered_sentence

filtered_sentence = processing(state_list)
print(filtered_sentence)
        
        

[['There', 'were', 'a', 'total', 'of', 'num1', 'soccer', 'games', 'during', 'the', 'num3', 'month', 'season'], ['Jill', 'invited', 'num3', 'people', 'to', 'her', 'birthday', 'party', 'They', 'each', 'ate', 'num2', 'pieces', 'of', 'pizza'], ['Tim', 'found', 'num3', 'seashells', 'and', 'Sally', 'found', 'num1', 'seashells', 'on', 'the', 'beach', 'When', 'they', 'cleaned', 'them', 'they', 'discovered', 'that', '25', 'were', 'cracked'], ['Kimberly', 'went', 'to', 'the', 'store', 'num3', 'times', 'last', 'month', 'She', 'buys', 'num2', 'peanuts', 'each', 'time', 'she', 'goes', 'to', 'the', 'store'], ['Wanda', 'has', 'num3', 'blocks', 'Theresa', 'gives', 'Wanda', 'num1', 'more'], ['Ezra', 'drew', 'a', 'white', 'line', 'that', 'was', 'num1', 'inches', 'long', 'Then', 'he', 'drew', 'a', 'blue', 'line', 'that', 'was', 'num3', 'inches', 'long'], ['Sarah', 'sold', 'num3', 'boxes', 'of', 'Lemon', 'Chalet', 'Cremes'], ['Joshua', 'has', 'num3', 'Skittles', 'and', 'eggs'], ['Willie', 'weighs', 'num2'

In [27]:

query_sentence = processing(query)
print(query_sentence)

[['If', 'the', 'games', 'are', 'equally', 'divided', 'how', 'many', 'soccer', 'games', 'are', 'played', 'a', 'month', '?', 'num1', 'num3', '/'], ['How', 'many', 'pieces', 'of', 'pizza', 'did', 'they', 'eat', '?', 'num3', 'num2', '*'], ['How', 'many', 'seashells', 'did', 'they', 'find', 'together', '?', 'num3', 'num1', '+'], ['How', 'many', 'peanuts', 'did', 'Kimberly', 'buy', 'last', 'month', '?', 'num3', 'num2', '*'], ['How', 'many', 'blocks', 'does', 'Wanda', 'have', 'in', 'all', '?', 'num3', 'num1', '+'], ['How', 'much', 'longer', 'was', 'the', 'white', 'line', 'than', 'the', 'blue', 'line', '?', 'num1', 'num3', '-'], ['How', 'many', 'cases', 'of', 'num2', 'boxes', 'plus', 'extra', 'boxes', 'does', 'Sarah', 'need', '?', 'num3', 'num2', '/'], ['If', 'he', 'shares', 'the', 'Skittles', 'among', 'num2', 'friends', 'how', 'many', 'Skittles', 'does', 'each', 'friend', 'get', '?', 'num3', 'num2', '/'], ['How', 'much', 'heavier', 'is', 'Willie', 'than', 'Kevin', '?', 'num2', 'num1', '-'], [

In [28]:
stop_words2 = ['num1','num2','num3','+','-','*','/','?'] ## to remove num1....

def extract_operator(query_sentence):
    operator=[]
    for idx in range(0,len(query_sentence)):
        operator.append(query_sentence[idx][-1])
        query_sentence[idx] = [w for  w in query_sentence[idx] if w not in stop_words2 ]
        
    return operator,query_sentence

operators,queries = extract_operator(query_sentence)
print(operators)
print(queries)   

['/', '*', '+', '*', '+', '-', '/', '/', '-', '*', '/', '+', '/', '-', '*', '-', '*', '+', '+', '+', '-', '+', '-', '-', '+', '/', '+', '+', '/', '-', '+', '*', '/', '+', '*', '+', '/', '+', '-', '-', '+', '-', '-', '*', '-', '-', '-', '*', '/', '+', '/', '-', '/', '+', '-', '-', '*', '+', '-', '+', '+', '-', '+', '+', '*', '+', '+', '*', '*', '*', '*', '/', '+', '+', '+', '+', '*', '-', '+', '*', '+', '+', '/', '+', '-', '-', '/', '/', '-', '-', '+', '-', '+', '+', '-', '-', '/', '+', '-', '-', '+', '+', '+', '-', '*', '-', '-', '/', '-', '-', '-', '+', '*', '*', '*', '*', '-', '-', '-', '/', '*', '-', '*', '*', '+', '/', '+', '/', '-', '-', '*', '-', '-', '+', '-', '/', '-', '-', '+', '/', '+', '*', '-', '+', '*', '/', '/', '-', '/', '*', '+', '/', '-', '-', '-', '+', '-', '-', '*', '*', '-', '-', '+', '-', '/', '*', '+', '*', '+', '+', '+', '-', '+', '-', '+', '+', '+', '+', '*', '+', '+', '-', '+', '+', '+', '-', '-', '/', '-', '-', '/', '/', '+', '/', '/', '-', '+', '+', '-', '-',

In [51]:
# encoded target variable
dic2={'+':1,'-':2,'*':3,'/':4}
def fun1(s):
    return dic2[s]

operator = [fun1(ops) for ops in operators]

In [52]:
print(operator)

[4, 3, 1, 3, 1, 2, 4, 4, 2, 3, 4, 1, 4, 2, 3, 2, 3, 1, 1, 1, 2, 1, 2, 2, 1, 4, 1, 1, 4, 2, 1, 3, 4, 1, 3, 1, 4, 1, 2, 2, 1, 2, 2, 3, 2, 2, 2, 3, 4, 1, 4, 2, 4, 1, 2, 2, 3, 1, 2, 1, 1, 2, 1, 1, 3, 1, 1, 3, 3, 3, 3, 4, 1, 1, 1, 1, 3, 2, 1, 3, 1, 1, 4, 1, 2, 2, 4, 4, 2, 2, 1, 2, 1, 1, 2, 2, 4, 1, 2, 2, 1, 1, 1, 2, 3, 2, 2, 4, 2, 2, 2, 1, 3, 3, 3, 3, 2, 2, 2, 4, 3, 2, 3, 3, 1, 4, 1, 4, 2, 2, 3, 2, 2, 1, 2, 4, 2, 2, 1, 4, 1, 3, 2, 1, 3, 4, 4, 2, 4, 3, 1, 4, 2, 2, 2, 1, 2, 2, 3, 3, 2, 2, 1, 2, 4, 3, 1, 3, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 2, 2, 4, 2, 2, 4, 4, 1, 4, 4, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 3, 1, 2, 3, 1, 2, 2, 2, 2, 1, 2, 3, 4, 3, 2, 1, 2, 4, 1, 1, 3, 2, 4, 3, 3, 4, 3, 1, 2, 1, 1, 2, 3, 2, 4, 2, 1, 3, 1, 2, 1, 1, 1, 1, 2, 1, 1, 3, 1, 4, 2, 3, 1, 3, 2, 2, 1, 4, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 3, 2, 2, 3, 1, 2, 4, 3, 3, 3, 1, 3, 1, 3, 2, 1, 2, 1, 2, 2, 2, 2, 3, 1, 4, 2, 1, 4, 2, 2, 1, 2, 2, 3, 4, 2, 3, 2, 3, 1, 4, 4, 1, 2, 1, 2, 2, 4, 3, 4, 4, 1, 2, 4, 1, 1, 2, 2, 

In [31]:
stemmer = PorterStemmer()

def list_to_string(lst):
    lst1 = []
    for i in range(0,len(lst)):
        singles = [stemmer.stem(plural) for plural in lst[i]]
        lst1.append(' '.join(singles))
    return lst1
        
query_sent = list_to_string(queries)
word_sent = list_to_string(filtered_sentence)
print(word_sent)


['there were a total of num1 soccer game dure the num3 month season', 'jill invit num3 peopl to her birthday parti they each ate num2 piec of pizza', 'tim found num3 seashel and salli found num1 seashel on the beach when they clean them they discov that 25 were crack', 'kimberli went to the store num3 time last month she buy num2 peanut each time she goe to the store', 'wanda ha num3 block theresa give wanda num1 more', 'ezra drew a white line that wa num1 inch long then he drew a blue line that wa num3 inch long', 'sarah sold num3 box of lemon chalet creme', 'joshua ha num3 skittl and egg', 'willi weigh num2 pound kevin weigh num1 pound', 'each ticket cost $ num20', '', 'there are num3 bird on the fenc num1 more bird land on the fenc', 'jack is invit num3 friend to a parti He ha num3 cooki', "alyssa 's high school play num1 hockey game thi year she attend num2 game", 'stephani went to the store num3 time last month she buy num1 orang each time she goe to the store', 'iren just bought 

In [32]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [33]:
vocab_size = 2000
tokenizer = Tokenizer(num_words=vocab_size, split=' ')
tokenizer.fit_on_texts(word_sent+query_sent)
word_numeric = tokenizer.texts_to_sequences(word_sent)
word_numeric=  pad_sequences(word_numeric)
query_numeric = tokenizer.texts_to_sequences(query_sent)
query_numeric=  pad_sequences(query_numeric)
print(query_numeric[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  25
   3  67   9 220 340   1   2 473  67   9 210  12 107]


In [34]:
max_len_word=len(word_numeric[100]) # this is numerical mapping of word sentence and vacabulary word
max_len_query=len(query_numeric[100])

In [35]:
RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 40

In [36]:
sentrnn = Sequential()
sentrnn.add(Embedding(vocab_size, EMBED_HIDDEN_SIZE,input_length=max_len_word))
sentrnn.add(Dropout(0.5))

qrnn = Sequential()
qrnn.add(Embedding(vocab_size, EMBED_HIDDEN_SIZE,input_length=max_len_query))
qrnn.add(Dropout(0.5))
qrnn.add(RNN(EMBED_HIDDEN_SIZE, return_sequences=False))
qrnn.add(RepeatVector(max_len_word))

model = Sequential()
model.add(Merge([sentrnn, qrnn], mode='sum'))
model.add(RNN(EMBED_HIDDEN_SIZE, return_sequences=False))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
#model.fit([X, Xq], Y, batch_size=BATCH_SIZE, nb_epoch=EPOCHS, validation_split=0.05)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if sys.path[0] == '':


In [ ]:
print('Build model...')
sentrnn = Sequential()
sentrnn.add(Embedding(vocab_size, EMBED_HIDDEN_SIZE,
                      input_length=max_len_word))
sentrnn.add(Dropout(0.3))
sentrnn.add(RNN(EMBED_HIDDEN_SIZE, return_sequences=False))
sentrnn.add(RepeatVector(max_len_word))
qrnn = Sequential()
qrnn.add(Embedding(vocab_size, EMBED_HIDDEN_SIZE,
                   input_length=max_len_query))
qrnn.add(Dropout(0.3))
qrnn.add(RNN(EMBED_HIDDEN_SIZE, return_sequences=False))
qrnn.add(RepeatVector(max_len_word))

model = Sequential()
model.add(Merge([sentrnn, qrnn], mode='sum'))
model.add(RNN(EMBED_HIDDEN_SIZE, return_sequences=True))
model.add(Dropout(0.3))
model.add(Dense(1, activation='softmax')) # as we have only 4 operators





In [37]:

model.fit([word_numeric,query_numeric], operator, batch_size=BATCH_SIZE, nb_epoch=EPOCHS, validation_split=0.05)

/opt/conda/lib/python3.6/site-packages/Keras-2.1.5-py3.6.egg/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 1247 samples, validate on 66 samples
Epoch 1/40
1247/1247 [==============================] - 6s 5ms/step - loss: 1.4072 - acc: 0.4186 - val_loss: 1.2676 - val_acc: 0.5000
Epoch 2/40
1247/1247 [==============================] - 5s 4ms/step - loss: 1.0681 - acc: 0.5389 - val_loss: 0.8981 - val_acc: 0.6818
Epoch 3/40
1247/1247 [==============================] - 5s 4ms/step - loss: 0.6465 - acc: 0.7698 - val_loss: 0.5856 - val_acc: 0.8182
Epoch 4/40
1247/1247 [==============================] - 5s 4ms/step - loss: 0.4192 - acc: 0.8645 - val_loss: 0.6082 - val_acc: 0.7879
Epoch 5/40
1247/1247 [==============================] - 5s 4ms/step - loss: 0.3069 - acc: 0.9038 - val_loss: 0.4325 - val_acc: 0.8030
Epoch 6/40
1247/1247 [==============================] - 5s 4ms/step - loss: 0.2051 - acc: 0.9447 - val_loss: 0.3741 - val_acc: 0.8788
Epoch 7/40
1247/1247 [==============================] - 5s 4ms/step - loss: 0.1290 - acc: 0.9623 - val_loss: 0.4485 - val_acc: 0.8182
Epoch 8/40
1247/

In [38]:
val = (word_numeric[0])


In [39]:
q = query_numeric[0]

In [40]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_1 (Merge)              (None, 43, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 255       
Total params: 240,655
Trainable params: 240,655
Non-trainable params: 0
_________________________________________________________________
None


In [41]:
loss, acc = model.evaluate([word_numeric, query_numeric], operator, batch_size=BATCH_SIZE)

1313/1313 [==============================] - 1s 569us/step


In [42]:
loss

0.017226401538927377

In [43]:
acc

0.9954303122619954

In [44]:
l=(model.predict([word_numeric[:], query_numeric[:]]))
m=[]
for i in range(0,len(l)):
    m.append(np.argmax(l[i]))
print (m)

[4, 3, 1, 3, 1, 2, 4, 4, 2, 3, 4, 1, 4, 2, 3, 2, 3, 1, 1, 1, 2, 1, 2, 2, 1, 4, 1, 1, 4, 2, 1, 3, 4, 1, 3, 1, 4, 1, 2, 2, 1, 2, 2, 3, 2, 2, 2, 3, 4, 1, 4, 2, 4, 1, 2, 2, 3, 1, 2, 1, 1, 2, 1, 1, 3, 1, 1, 3, 3, 3, 3, 4, 1, 1, 1, 1, 3, 2, 1, 3, 1, 1, 4, 1, 2, 2, 4, 4, 2, 2, 1, 2, 1, 1, 2, 2, 4, 1, 2, 2, 1, 1, 1, 2, 3, 2, 2, 4, 2, 2, 2, 1, 3, 3, 3, 3, 2, 2, 2, 4, 3, 2, 3, 3, 1, 4, 1, 4, 2, 2, 3, 2, 2, 1, 2, 4, 2, 2, 1, 4, 1, 3, 2, 1, 3, 4, 4, 2, 4, 3, 1, 4, 2, 2, 2, 1, 2, 2, 3, 3, 2, 2, 1, 2, 4, 3, 1, 3, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 2, 2, 4, 2, 2, 4, 4, 1, 4, 4, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 3, 1, 2, 3, 1, 2, 2, 2, 2, 1, 2, 3, 4, 3, 2, 1, 2, 4, 1, 1, 3, 2, 4, 3, 3, 4, 3, 1, 2, 1, 1, 2, 3, 2, 4, 2, 1, 3, 1, 2, 1, 1, 1, 1, 2, 1, 1, 3, 1, 4, 2, 3, 1, 3, 2, 2, 1, 4, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 3, 2, 2, 3, 1, 2, 4, 3, 3, 3, 1, 3, 1, 3, 2, 1, 2, 1, 2, 2, 2, 2, 3, 1, 4, 2, 1, 4, 2, 2, 1, 2, 2, 3, 4, 2, 3, 2, 3, 1, 4, 4, 1, 2, 1, 2, 2, 4, 3, 4, 4, 1, 2, 4, 1, 1, 2, 2, 

In [45]:

def fun(l1,l2):
    print(len(l1))
    print(type(l2))
    count = 0
    for i in range(0,len(l1)):
        if(l1[i]==l2[i]):
            count = count+1
    return count,len(l1)

a,b = fun(m,operator)
print(a/b)
print(b)

1313
<class 'list'>
0.9954303122619954
1313


In [64]:

def convert_to_d(test):
    val_test = extract(test)
    query_test,statement_test = extract_query(val_test)
    state_list_test = clean_data(statement_test)
    filtered_sentence_test = processing(state_list_test)
    query_sentence_test = processing(query_test)
    operators_test,queries_test = extract_operator(query_sentence_test)
    operator_test = [fun1(ops) for ops in operators_test]
    query_sent_test = list_to_string(queries_test)
    word_sent_test = list_to_string(filtered_sentence_test)
    word_numeric_test = tokenizer.texts_to_sequences(word_sent_test)
    word_numeric_test=  pad_sequences(word_numeric_test,max_len_word)
    query_numeric_test = tokenizer.texts_to_sequences(query_sent_test)
    query_numeric_test=  pad_sequences(query_numeric_test,max_len_query)
    return word_numeric_test,query_numeric_test,operator_test
print (test)
word_numeric_test,query_numeric_test,operator_test = convert_to_d(test)
print(word_numeric_test)
b=(model.predict([word_numeric_test[:], query_numeric_test[:]]))
print(b)


    

<_io.TextIOWrapper name='../input/testfile/test.txt' mode='r' encoding='UTF-8'>
[[  0   0   0 ...  10   4  86]
 [  0   0   0 ...   7   3 305]
 [  0   0   0 ...   7   3 372]
 ...
 [  0   0   0 ...  10   5  32]
 [  0   0   0 ... 394  10   4]
 [  0   0   0 ... 736  10   6]]
[[1.2500596e-04 9.9486536e-01 4.6679988e-03 2.8039221e-04 6.1257626e-05]
 [1.2749074e-06 3.7288537e-05 9.9991751e-01 1.0379459e-05 3.3473432e-05]
 [1.2712430e-06 2.9305616e-05 9.9992788e-01 1.0785994e-05 3.0781401e-05]
 ...
 [2.1288561e-05 9.9969935e-01 2.2929984e-04 4.1264229e-05 8.8315619e-06]
 [7.2561170e-07 1.0051339e-04 9.9987471e-01 5.7417174e-06 1.8402254e-05]
 [1.8155832e-05 9.9976772e-01 1.5512240e-04 5.2749758e-05 6.1425158e-06]]


In [62]:
for line in test:
    print(line)

In [65]:
# for test set

m1=[]
for i in range(0,len(b)):
    m1.append(np.argmax(b[i]))
print (m1)

[1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 4, 1, 2, 2, 2, 1, 2, 1, 2, 4, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 3, 4, 4, 4, 4, 4, 2, 1, 3, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 3, 4, 4, 4, 2, 3, 3, 3, 1, 1, 4, 1, 3, 1, 3, 4, 3, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 3, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 3, 3, 4, 4, 4, 3, 4, 2, 1, 2, 2, 4, 4, 2, 2, 3, 3, 3, 2, 2, 3, 4, 4, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 4, 4, 3, 3, 3, 4, 4, 4, 4, 3, 1, 3, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 3, 2, 1, 1, 3, 2, 2, 2, 1, 2, 3, 3, 1, 3, 3, 3, 3, 1, 2, 2, 2, 2, 3, 1, 2, 2, 3, 3, 3, 3, 3, 1, 3, 4, 2, 2, 4, 2, 4, 4, 2, 2, 2, 2, 1, 

In [66]:
a1,b1 = fun(m1,operator_test)
print(a1/b1)
print(b1)

437
<class 'list'>
0.8947368421052632
437
